In [9]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import random
import itertools
from apyori import apriori

# import pywt

# import pymultifracs.mfa as mfa
# from pymultifracs.utils import build_q_log
# from statsmodels.tsa.ar_model import AutoReg, ar_select_order


In [10]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

In [11]:

# from sklearn.model_selection import  KFold

In [12]:
# %pip install import_ipynb
# %pip install  --user git+https://github.com/neurospin/pymultifracs

In [13]:
import import_ipynb
#from transformations import DataTransform,TransformationRegistry, IdentityTransform, FourierTransform, LowFourierTransform, LowPsdTransform, WaveDecTransform, DwtTransform, CwtTransform, AutoRegTransform, ShannonEncodingTransform, WaveletLeadersTransform, CrossCorTransform, AutoCorTransform, MultiFracsTransform, AutoEncoderTransform  
from transformations import *
from prep_synth_data import create_synth_data

In [14]:
# Initialize the registry
registry = initialize_registry()#TransformationRegistry()


In [36]:
hrv_data = np.load('med_hrv_data.npy')
ecg_data = np.load('med_ecg_data.npy')

X_hrv, y_hrv = hrv_data[:,:-1], hrv_data[:,-1]
X_ecg, y_ecg = ecg_data[:,:-1], ecg_data[:,-1]

hrv_data = np.load('better_hrv_data.npy')
ecg_data = np.load('better_ecg_data.npy')

X_hrv, y_hrv = hrv_data[:,:-1], hrv_data[:,-1]
X_ecg, y_ecg = ecg_data[:,:-1], ecg_data[:,-1]

In [37]:
def evaluate_transformations(X, y, transformations, classifiers):
    dt = DataTransform(registry)
    results = []

    for trans in transformations:
        trans_name_str = DataTransform.get_full_trans_kwargs_str(trans)
        trans_X = dt.apply_transformation(X, trans)

        result = {'Transformation (shape)': trans_name_str}
        result["shape"] = trans_X.shape[-1]
        result['nb_trans'] = len(trans)

        scaler = StandardScaler()
        trans_X = scaler.fit_transform(trans_X) 
        print(f"Transformation: {trans_name_str}, Shape: {trans_X.shape[-1]} ")
        for clf_name, clf in classifiers.items():
            kf = KFold(n_splits=5, shuffle=True, random_state=42)
            scores = cross_val_score(clf, trans_X, y, cv=kf, scoring='accuracy')
            result[f'{clf_name} accuracy'] = np.mean(scores)
            result[f'{clf_name} std'] = np.std(scores)
            print(f"Classifier: {clf_name}, "
                  f"Mean Accuracy: {np.mean(scores):.3f}, Std Accuracy: {np.std(scores):.3f}")
        
        results.append(result)

    return pd.DataFrame(results)

In [41]:
transformations = [[['multifracs',{'j1':2,'j2':10}],['autoreg',{'k':5}], 'shannon_encoding'],'waveletscattering',['waveletscattering',['multifracs',{'j1':2,'j2':10}],['autoreg',{'k':5}], 'shannon_encoding']]

classifiers = {
    'SVM': SVC(kernel='linear'),
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier(),
    'MLPClassifier': MLPClassifier(max_iter=500),
}

In [42]:
results_df = evaluate_transformations(X_hrv, y_hrv, transformations, classifiers)

Computing  multifracs_j1=2_j2=10 ...

c:\users\ant\documents\info2\bits2beat-tests\pymultifracs\pymultifracs\utils.py:87: RuntimeWarning: divide by zero encountered in power
  return np.power(array, exponent)
c:\users\ant\documents\info2\bits2beat-tests\pymultifracs\pymultifracs\utils.py:76: RuntimeWarning: divide by zero encountered in reciprocal
  return array ** exponent
c:\users\ant\documents\info2\bits2beat-tests\pymultifracs\pymultifracs\cumulants.py:265: RuntimeWarning: divide by zero encountered in log
  log_T_X_j = np.log(T_X_j)
c:\users\ant\documents\info2\bits2beat-tests\pymultifracs\pymultifracs\mfspectrum.py:117: RuntimeWarning: invalid value encountered in divide
  R_j = temp / Z
c:\users\ant\documents\info2\bits2beat-tests\pymultifracs\pymultifracs\mfspectrum.py:118: RuntimeWarning: divide by zero encountered in log2
  V[:, ind_j, :] = fixednansum(R_j * np.log2(mrq_values_j), axis=1)
c:\users\ant\documents\info2\bits2beat-tests\pymultifracs\pymultifracs\mfspectrum.py:118: RuntimeWarning: invalid value encoun

Transformation: multifracs_j1=2_j2=10_autoreg_k=5_shannon_encoding, Shape: 16 
Classifier: SVM, Mean Accuracy: 0.965, Std Accuracy: 0.018
Classifier: DecisionTree, Mean Accuracy: 0.972, Std Accuracy: 0.020
Classifier: RandomForest, Mean Accuracy: 0.967, Std Accuracy: 0.023
Classifier: MLPClassifier, Mean Accuracy: 0.968, Std Accuracy: 0.015
Transformation: waveletscattering, Shape: 78150 
Classifier: SVM, Mean Accuracy: 0.922, Std Accuracy: 0.035
Classifier: DecisionTree, Mean Accuracy: 0.835, Std Accuracy: 0.027
Classifier: RandomForest, Mean Accuracy: 0.900, Std Accuracy: 0.026
Classifier: MLPClassifier, Mean Accuracy: 0.945, Std Accuracy: 0.027


In [43]:
results_df = evaluate_transformations(X_ecg, y_ecg, transformations, classifiers)

Computing  multifracs_j1=2_j2=10 ...

c:\users\ant\documents\info2\bits2beat-tests\pymultifracs\pymultifracs\utils.py:87: RuntimeWarning: divide by zero encountered in power
  return np.power(array, exponent)
c:\users\ant\documents\info2\bits2beat-tests\pymultifracs\pymultifracs\utils.py:76: RuntimeWarning: divide by zero encountered in reciprocal
  return array ** exponent
c:\users\ant\documents\info2\bits2beat-tests\pymultifracs\pymultifracs\cumulants.py:265: RuntimeWarning: divide by zero encountered in log
  log_T_X_j = np.log(T_X_j)
c:\users\ant\documents\info2\bits2beat-tests\pymultifracs\pymultifracs\mfspectrum.py:117: RuntimeWarning: invalid value encountered in divide
  R_j = temp / Z
c:\users\ant\documents\info2\bits2beat-tests\pymultifracs\pymultifracs\mfspectrum.py:118: RuntimeWarning: divide by zero encountered in log2
  V[:, ind_j, :] = fixednansum(R_j * np.log2(mrq_values_j), axis=1)
c:\users\ant\documents\info2\bits2beat-tests\pymultifracs\pymultifracs\mfspectrum.py:118: RuntimeWarning: invalid value encoun

Transformation: multifracs_j1=2_j2=10_autoreg_k=5_shannon_encoding, Shape: 16 
Classifier: SVM, Mean Accuracy: 0.930, Std Accuracy: 0.019
Classifier: DecisionTree, Mean Accuracy: 0.910, Std Accuracy: 0.040
Classifier: RandomForest, Mean Accuracy: 0.938, Std Accuracy: 0.027


c:\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Classifier: MLPClassifier, Mean Accuracy: 0.950, Std Accuracy: 0.026
Computing  waveletscattering ...

c:\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Transformation: waveletscattering, Shape: 156250 
Classifier: SVM, Mean Accuracy: 0.955, Std Accuracy: 0.024
Classifier: DecisionTree, Mean Accuracy: 0.825, Std Accuracy: 0.049
Classifier: RandomForest, Mean Accuracy: 0.915, Std Accuracy: 0.024
Classifier: MLPClassifier, Mean Accuracy: 0.950, Std Accuracy: 0.032
